In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
# selenium으로 무엇인가 입력하기 위한 import
from selenium.webdriver.common.keys import Keys
import time

import pandas as pd
import re
import os

In [59]:
# 크롬드라이버 위치 절대경로로 설정
driver = webdriver.Chrome('/Users/yangsemo/chromedriver')
driver.implicitly_wait(10) # seconds
page = 1
url = f'https://myfranchise.kr/category/contents/%EC%B9%B4%ED%8E%98%C2%B7%EB%94%94%EC%A0%80%ED%8A%B8?page={page}'
driver.get(url)



/var/folders/xy/wq52q6wn4ts_3hz5xm2fj5_80000gn/T/ipykernel_2751/2950028987.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/yangsemo/chromedriver')


In [60]:
x = 1 # page down을 위함
page = 1

# dataframe 생성
col = ['브랜드명','매장수','상호', '대표자', '법인설립등기일', '사업자등록일', '대표번호', '대표팩스번호', '사업자유형', '법인등록번호', '사업자등록번호', '주소']
df = pd.DataFrame(columns=col)

# 한페이지에 나와있는 카페 수
cafe_len = len(driver.find_elements(By.CLASS_NAME, "css-h9u7nh"))
print('cafe 개수: ', cafe_len)

while(True):
    try:
        print('if문 확인')
        if x == cafe_len+1:
            x=1
            page+=1
            
            print('------------page 변경--------------')
            print('page: ', page)
            print('----------------------------------')
            
            url = f'https://myfranchise.kr/category/contents/%EC%B9%B4%ED%8E%98%C2%B7%EB%94%94%EC%A0%80%ED%8A%B8?page={page}'
            driver.get(url)
            time.sleep(2)
            
            # 한페이지에 나와있는 카페 수
            # 마지막 페이지 카페 수는 7개 이므로 cafe_len 갱신 필요
            cafe_len = len(driver.find_elements(By.CLASS_NAME, "css-h9u7nh"))
            print('Next Page cafe 개수: ', cafe_len)

            if page == 56:
                break

        else:
            for i in range(cafe_len):
                
                # 본사 정보 Data List
                val = []
                
                # 브랜드명
                cafe = driver.find_elements(By.CLASS_NAME, "css-1w2bh6w")[i]

                cafe_name = cafe.find_element(By.CLASS_NAME, "css-1xkzyxt")
                cafe_count = cafe.find_element(By.CLASS_NAME, "css-1wz331c")

                cafe_name_text = cafe_name.get_attribute("innerText")
                
                cafe_count_text1 = cafe_count.get_attribute("innerText").split(' ')[1]
                cafe_count_text2 = re.sub(r'[^0-9]', '', cafe_count_text1)
                
                
                val.append(cafe_name_text)
                val.append(cafe_count_text2)
                print('번호:', x, ' 브랜드명:',cafe_name_text, ' 매장수:',cafe_count_text2)
                cafe.click()
                
                time.sleep(1)
                

                # 본사 탭 클릭
                bonsa = driver.find_element('xpath', '//*[@id="brand-relative-tabs-mobile"]/div[1]/div/div')
                length = len(bonsa.find_elements(By.TAG_NAME, 'button'))
                bonsa_btn = bonsa.find_elements(By.TAG_NAME, 'button')[-1]
                bonsa_btn.click()
                
                time.sleep(1)

                
                # 본사 정보                
                company = driver.find_element(By.CLASS_NAME, 'css-p43zmh')


                for val_name in company.find_elements(By.CLASS_NAME, "css-26ughq"):
                    val.append(val_name.get_attribute("innerText"))
                    
                # 주소만 class_name이 다르므로 따로 추가
                address = company.find_element(By.CLASS_NAME, "css-m8flp7")
                address_name = address.get_attribute('innerText')
                val.append(address_name)
            
                # 카페 인덱스 증가
                x+=1
                
                # 데이터 삽입
                df.loc[len(df)] = val

                driver.back()
                time.sleep(1)

    # 예외처리로 데이터 못 불러오면 pagedown
    except Exception as error:
        print('error:', error)
        body = driver.find_element(By.CSS_SELECTOR,'body')
        body.send_keys(Keys.PAGE_DOWN)

        
df.to_csv('cafe_info.csv', index=False)

cafe 개수:  24
if문 확인
번호: 1  브랜드명: 미콩커피  매장수: 8
번호: 2  브랜드명: 컵넛(cupnut)  매장수: 23
번호: 3  브랜드명: 고더커피  매장수: 8
번호: 4  브랜드명: 오늘도, 카페일리터  매장수: 96
번호: 5  브랜드명: 영커피  매장수: 39
번호: 6  브랜드명: 사생활  매장수: 8
번호: 7  브랜드명: 달다곰이  매장수: 35
번호: 8  브랜드명: 청솔로9  매장수: 61
번호: 9  브랜드명: 더플롯커피(THE PLOT COFFEE)  매장수: 9
번호: 10  브랜드명: 몬스터커피  매장수: 65
번호: 11  브랜드명: 욤카페  매장수: 14
번호: 12  브랜드명: 크레페  매장수: 5
번호: 13  브랜드명: 댄싱컵(DANCINGCUP)  매장수: 63
번호: 14  브랜드명: 디커피  매장수: 2
번호: 15  브랜드명: 카페게이트  매장수: 133
번호: 16  브랜드명: 쿼드커피(QUAD COFFEE)  매장수: 17
번호: 17  브랜드명: 루시카토 베이크카페  매장수: 17
번호: 18  브랜드명: 꽈페  매장수: 15
번호: 19  브랜드명: 카페우드진  매장수: 4
번호: 20  브랜드명: 트리플에이커피  매장수: 53
번호: 21  브랜드명: 캘리포니아도넛클럽  매장수: 7
번호: 22  브랜드명: 타타스베이글  매장수: 7
번호: 23  브랜드명: 요고 프로즌요거트(YOGO FROZEN YOGURT)  매장수: 5
번호: 24  브랜드명: 이삐커피(yippee coffee)  매장수: 13
if문 확인
------------page 변경--------------
page:  1
----------------------------------
Next Page cafe 개수:  24
if문 확인
번호: 1  브랜드명: 커피 로드뷰  매장수: 3
번호: 2  브랜드명: 핑크트럭 츄러스  매장수: 6
번호: 3  브랜드명: 카페마일로  매장수: 15
번호: 4  브랜드명: 아아수혈  

KeyboardInterrupt: 

In [61]:
df

,브랜드명,매장수,상호,대표자,법인설립등기일,사업자등록일,대표번호,대표팩스번호,사업자유형,법인등록번호,사업자등록번호,주소
0,미콩커피,8,미콩커피본사,조항성,-,2020-11-12,001-577-0541,000000000,개인,-,628-18-01312,우 : 34553 대전광역시 동구 홍도로 14 201호(홍도동)
1,컵넛(cupnut),23,컵넛(cupnut),박근민,-,2019-01-03,051-1577-7381,-,개인,-,340-03-02455,우 : 46272 부산광역시 금정구 부곡로 133-15 1층 cupnut
2,고더커피,8,고더커피,심재용,-,2021-07-15,051-555-7716,-,개인,-,602-48-00642,"우 : 47736 부산광역시 동래구 온천천로 165 101동 121호(명륜동, 동래..."
3,"오늘도, 카페일리터",96,카페일리터,한초롱,-,2016-01-18,016700170,000,개인,-,825-11-00235,우 : 10424 경기도 고양시 일산동구 호수로430번길 7-9 1층(백석동)
4,영커피,39,(주)영투게더컴퍼니,장영생,2019-03-22,2019-03-22,051-1811-7590,051-783-0199,법인,180111-1208297,603-87-01395,우 : 48051 부산광역시 해운대구 재반로165번길 121 1층
5,사생활,8,(주)화합과전진,"김성훈, 심영근",2021-02-01,2021-02-01,070-5033-7759,000-0000-0000,법인,110111-7768744,353-87-02000,"우 : 08375 서울특별시 구로구 디지털로31길 41 312-207호 (구로동, ..."
6,달다곰이,35,씨앰비즈(주),서보람,2020-11-13,2020-12-01,1577-8149,-,법인,110111-7683380,591-86-01891,우 : 06030 서울특별시 강남구 논현로175길 40 지상1층 102호
7,청솔로9,61,청솔로9가맹본부,김태우,-,2019-06-24,1666-8663,070-7543-6466,개인,-,474-09-01090,"우 : 42162 대구광역시 수성구 청수로 62, 6층(상동)"
8,더플롯커피(THE PLOT COFFEE),9,더플롯(THEPLOT),이영주,-,2014-09-20,070-8840-1884,050-4082-0663,개인,-,109-15-73575,"우 : 07572 서울특별시 강서구 공항대로45길 7 (등촌동, 1층)"
9,몬스터커피,65,㈜몬스터,안기수,2019-09-25,2019-09-25,053-313-3683,000-0000-0000,법인,170111-0739138,490-81-01540,우 : 41422 대구광역시 북구 팔거천동로 217 104호(동천동)
